In [ ]:
%pip install fastapi uvicorn openai requests
from IPython.display import clear_output ; clear_output()

In [ ]:
%pycat aoai_proxy_config.py

In [ ]:
%pycat aoai_proxy_server.py

In [ ]:
import os
from time import sleep

os.system("""uvicorn aoai_proxy_server:app &""")
sleep(3)

In [ ]:
import openai
import requests

from aoai_proxy_config import USERS

openai.api_type = 'azure'
openai.api_version = '2023-03-15-preview'
openai.api_base = 'http://127.0.0.1:8000'

deployment_id = 'gpt-35-turbo' # Replace if using a different deployment name

def chat(user, prompt):
    openai.api_key = USERS[user]['api_key']
    completion = openai.ChatCompletion.create(
        deployment_id='gpt-35-turbo',
        messages=[{'role': 'user', 'content': prompt}],
    )
    print(f'{user}: {prompt}')
    print(completion.choices[0]['message']['content'])
    print('---------------')
    usage = requests.get('http://127.0.0.1:8000/usage').json()
    print(f'Total cost: ${usage["total_cost"]:.7f}')
    for user, user_usage in usage['users'].items():
        print(f'{user} cost: ${user_usage["total_cost"]:.7f}')
    for endpoint, endpoint_usage in usage['endpoints'].items():
        print(f'{endpoint} cost: ${endpoint_usage["total_cost"]:.7f}')

In [ ]:
chat('Angela', 'What is the capital of France?')

In [ ]:
chat('Benjamin', 'Write a haiku about rainbows.')

In [ ]:
chat('Cynthia', 'Count from 1 to 23.')

In [ ]:
chat('David', 'What are 7 words starting with the letter "a"?')

In [ ]:
chat('Angela', 'Who was the 11th president of the US?')

In [ ]:
chat('Cynthia', 'What number comes after 17?')

In [ ]:
os.system("""kill $(ps aux | grep '[u]vicorn' | awk '{print $2}')""")